In [9]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

import pickle

ImportError: cannot import name 'show_config' from 'numpy' (unknown location)

In [10]:
import numpy

In [8]:
!numpy --v

/bin/bash: numpy: command not found


In [2]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Dropout, InputLayer, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import model_from_json
import numpy as np
import pandas as pd

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

In [3]:
from tensorflow.math import exp

In [19]:
with open('dataset6_{}.pkl'.format('features'), 'rb') as fp:
    dataset = pickle.load(fp)

path_to_save = "/scratch/cinthiasouza/mv-text-summarizer/notebook/models_v6"

In [5]:
def save_nn(model, name, path_to_save):
    
    model_json = model.to_json()

    with open("{}/mlp_{}.json".format(path_to_save, name), "w") as json_file:
        json_file.write(model_json)

    model.save_weights("{}/mlp_{}.h5".format(path_to_save, name))
    

In [6]:
def scheduler(epoch, lr):
    if epoch < 20:
        return lr
    else:
        return lr * exp(-0.1)

In [7]:
from tensorflow.keras.callbacks import Callback

class TerminateOnBaseline(Callback):
    """Callback that terminates training when either acc or val_acc reaches a specified baseline
    """
    def __init__(self, monitor='accuracy', baseline=0.9):
        super(TerminateOnBaseline, self).__init__()
        self.monitor = monitor
        self.baseline = baseline

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        acc = logs.get(self.monitor)
        if acc is not None:
            if acc >= self.baseline:
                print('Epoch %d: Reached baseline, terminating training' % (epoch))
                self.model.stop_training = True

In [20]:
parameters_mlp = { 
    'introduction':  [ 0.2, 200, 8],
    'materials':     [ 0.2, 200, 8],
    'conclusion':  [ 0.2, 200, 8]}

In [21]:
section='materials'

X = dataset[section][0]
y = dataset[section][2]

model = Sequential()
model.add(Dense(128, input_dim=X.shape[1], activation='relu'))
model.add(Dropout(.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(.2))
model.add(Dense(2, activation='sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(
                learning_rate=0.001), metrics=[keras.metrics.Precision()])

one_hot_label = to_categorical(y)

test_size, epochs, batch_size =parameters_mlp.get(section)

X_train, X_valid, y_train, y_valid = train_test_split(X, one_hot_label, stratify=one_hot_label, test_size=test_size)

my_callbacks = [EarlyStopping(patience=20)]

my_callbacks = [EarlyStopping(patience=10, restore_best_weights=True),
                keras.callbacks.LearningRateScheduler(scheduler),
                TerminateOnBaseline(monitor='val_precision', baseline=0.9)]


model.fit(
            np.array(X_train), np.array(y_train), validation_data=(X_valid,y_valid), epochs=epochs,
             batch_size=batch_size, class_weight={0:6, 1:1}, callbacks=my_callbacks, verbose=True)


Epoch 1/200
7379/7379 [==============================] - 235s 24ms/step - loss: 1.1919 - precision_2: 0.5971 - val_loss: 0.7498 - val_precision_2: 0.6311
Epoch 2/200
7379/7379 [==============================] - 200s 27ms/step - loss: 1.1325 - precision_2: 0.5950 - val_loss: 0.7468 - val_precision_2: 0.5548
Epoch 3/200
7379/7379 [==============================] - 117s 16ms/step - loss: 1.1224 - precision_2: 0.5747 - val_loss: 0.7703 - val_precision_2: 0.5949
Epoch 4/200
7379/7379 [==============================] - 221s 30ms/step - loss: 1.1205 - precision_2: 0.5841 - val_loss: 0.8312 - val_precision_2: 0.5527
Epoch 5/200
7379/7379 [==============================] - 197s 27ms/step - loss: 1.1167 - precision_2: 0.5696 - val_loss: 0.7720 - val_precision_2: 0.5894
Epoch 6/200
7379/7379 [==============================] - 116s 16ms/step - loss: 1.1110 - precision_2: 0.5834 - val_loss: 0.7643 - val_precision_2: 0.5995
Epoch 7/200
7379/7379 [==============================] - 123s 17ms/step - lo

In [114]:
X_test=dataset[section][1]
y_test =one_hot_label = to_categorical(dataset[section][3])


In [22]:
model_json = model.to_json()
with open('{}/mlp_{}.json'.format(path_to_save, section), "w") as json_file:
      json_file.write(model_json)
model.save_weights('{}/mlp_{}.h5'.format(path_to_save, section))
print("Saved model to disk")

Saved model to disk


In [115]:
model.evaluate(X_test, y_test)

982/982 [==============================] - 10s 9ms/step - loss: 0.2994 - precision_27: 0.8786


[0.29938286542892456, 0.8786105513572693]